Please report any problem under Issues in my Github repository: https://github.com/Ethanleet/ECE199

http://www.jmlr.org/proceedings/papers/v40/Kamath15.pdf

In [1]:
import ipywidgets as widgets
import numpy as np
import plotly
from IPython.display import display
from ipywidgets import Output, FloatSlider
from ipywidgets import *
from plotly.graph_objs import *

plotly.offline.init_notebook_mode()
widget_output = Output()

In [2]:
'''
Description: Integrated loss function. Computes L1 distance,
             L2 squared distance, Chi-squared divergence and
             KL divergence.
Param1: p is the observed value
Param2: q is the predicted value
Param3: funcs specifies which loss functions are used
Return: Vector of results from different loss functions
'''
def loss(p, q, funcs):
    p = np.array(p)
    q - np.array(q)
    ret_val = []
    for func in funcs:
        if func == "L1":
            # L1 distance
            ret_val.append(np.sum(np.absolute(p - q)))
        elif func == "L2_sq":
            # L2 squared distance
            ret_val.append(np.sum(np.power((p - q), 2)))
        elif func == "chi_sq":
            # chi-squared divergence
            ret_val.append(np.sum(np.power((p - q), 2) / q))
        elif func == "KL_div":
            # KL-divergence
            ret_val.append(np.sum(p * np.log(p / q)))
        else:
            raise ValueError("Please verify your loss function is valid")
    return ret_val

In [3]:
'''
Description: Generates a uniform distribution
Param1: n is the number of symbols
Return: vector containing probability for each symbol
'''
def uniform(n):
    return [1/n]*n

'''
Description: Generates a zipf distribution with offset = 0
Param1: n is the number of symbols
Param2: a is the parameter for zipf distribution
Return: vector containing probability for each symbol
'''
def zipf(n, a):
    p = [0]*n
    sum = 0
    for i in range (0, n):
        p[i] = 1 / (np.power(i + 1, a))
        sum += p[i]
    return p / sum

'''
Description: Generates a random sample set with given distribution
Param1: n is number of samples
Param2: p is the distribution/probability vector
Return: vector containing how many times each symbol appears
'''
def sample_gen(n, p):
    # Generate random numbers in sample space
    samples = np.sort(np.random.uniform(0,1,n))
    symbol_cnt = np.zeros(len(p), dtype=int) # initialize count for each symbol
    
    if debug: print ("[DEBUG] Samples:", samples)
    
    # Assign & count symbols
    # Separate sorted radom sample from 0 to 1 with the given probability distribution
    # Optimized for two cases where number of samples, or number of symbols is greater
    done = False
    if n < len(p):
        sample_index = 0
        symbol_val = 0
        for symbol_index in range(0, len(p)):
            if done: break
            symbol_val += p[symbol_index]
            while samples[sample_index] < symbol_val:
                symbol_cnt[symbol_index] += 1
                if sample_index < n - 1:
                    sample_index += 1
                else:
                    done = True
                    break
    else:
        symbol_index = 0
        symbol_val = p[symbol_index]
        for sample_index in range(0, n):
            if done: break
            while symbol_val < samples[sample_index]:
                if symbol_index < len(p) - 1:
                    symbol_index += 1
                    symbol_val += p[symbol_index]
                else:
                    done = True
                    break
            symbol_cnt[symbol_index] += 1
            
    if debug: print ("[DEBUG] Symbol distribution:", symbol_cnt)
    return symbol_cnt


'''
Description: Runs a selected estimator. Supports add-constant,
             Braess and Sauer, and Good-Turing estimators.
Param1: n is number of samples
Param2: p is the distribution/probability vector
Param3: symbol_cnt shows how many times each symbol appears
Param4: estimator should be specified as "Add-constant", "Braess-Sauer" or "Good-Turing
Param5: Specific parameters for some estimators. For add-constant estimator, this should
        be beta value.
Return: The estimated distribution q, as a vector
'''
def estimate(n, p, symbol_cnt, estimator, params=0):
    k = len(p) # number of symbols
    q = np.empty(k, dtype=float)
    
    # Add-constant estimator
    if estimator == "Add-constant":
        beta = params
        for i in range(0, k):
            q[i] = (symbol_cnt[i] + beta) / (n + k * beta)
    
    # Braess and Sauer estimator
    elif estimator == "Braess-Sauer":
        for i in range(0, k):
            if symbol_cnt[i] == 0:
                beta = 0.5
            elif symbol_cnt[i] == 1:
                beta = 1
            else:
                beta = 0.75
            q[i] = (symbol_cnt[i] + beta) / (n + k * beta)
    
    # Good-Turing estimator
    elif estimator == "Good-Turing":
        # Vector of phi's where phi_t denotes number of elements appearing t times
        # t can vary from 0 to n+1, as phi_{t+1} is also used by this estimator
        phi = np.zeros(n+2, dtype=int)
        for t in symbol_cnt:
            phi[t] += 1
            
        # This step is modified from the original formula, as it does not divide q
        # by the normalization factor N, which will be done later.
        for i in range(0, k):
            t = symbol_cnt[i]
            if t > phi[t+1]:
                q[i] = t
            else:
                q[i] = (phi[t+1] + 1) * (t + 1) / phi[t]
                
        # Divide all probabilities by normalization factor to ensure they add up to 1
        q = [x / sum(q) for x in q]
        
    else:
        raise ValueError("Please verify your estimator is valid")
    
    if debug and estimator != "Add-constant": print("[DEBUG]", estimator, "estimator:", q)
    return q

In [4]:
'''
Description: Generate random samples, estimate, and calculate error
             for add-constant estimator
Param1: n is number of samples
Param2: p is the distribution/probability vector
Param3: beta_arr is the range of beta for add-constant estimator
Param4: maxIterations is the number of iterations to run
Return: A list containing mean and standard deviation for errors
'''
def run_add_constant (n, p, beta_arr, maxIterations):
    k = len(p) # number of symbols
    error_list = [] # store error matrices across iterations
    error_matrix_sum = 0 # sum of error matrices across iterations
    
    for iteration in range(0, maxIterations):
        if debug and iteration: print("[DEBUG]")
        if debug: print("[DEBUG] Data set", iteration)
        symbol_cnt = sample_gen(n, p)
        
        # Add-constant estimator for each beta value
        error_matrix = [] # last two elements are from BS and GT estimators
        for beta in beta_arr:
            q = estimate(n, p, symbol_cnt, "Add-constant", beta)
            error = loss(p, q, ["L1", "L2_sq", "chi_sq", "KL_div"])
            error_matrix.append(error)
        
        # Braess and Sauer estimator
        q = estimate(n, p, symbol_cnt, "Braess-Sauer")
        error = loss(p, q, ["L1", "L2_sq", "chi_sq", "KL_div"])
        error_matrix.append(error)
        
        # Good-Turing estimator
        q = estimate(n, p, symbol_cnt, "Good-Turing")
        error = loss(p, q, ["L1", "L2_sq", "chi_sq", "KL_div"])
        error_matrix.append(error)
        
        # Calculate error mean over iterations
        error_matrix = np.asarray(error_matrix)
        error_list.append(error_matrix)
        error_matrix_sum += error_matrix
    error_matrix_mean = error_matrix_sum / maxIterations
    
    # Calculate error standard deviation over iterations
    error_matrix_sqsum = 0
    for iteration in range(0, len(error_list)):
        error_matrix_sqsum += np.square(error_list[iteration] - error_matrix_mean)
    error_matrix_std_dev = np.sqrt(error_matrix_sqsum / maxIterations)
    
    if debug: print ("[DEBUG] Error_matrix dimensions:", error_matrix_sum.shape)
    
    # returns a list consists of error matrix mean and its standard deviation
    # each row corresponds to a loss function, each col corresponds to a beta value
    return [error_matrix_mean.T, error_matrix_std_dev.T]

In [5]:
'''
Description: Generate random samples, estimate, and calculate error
             for both Braess Sauer and Good-Turing estimators
Param1: n is number of samples
Param2: p is the distribution/probability vector
Param3: mode indicates if this function estimates with Braess Sauer or Good-Turing estimator
Param4: maxIterations is the number of iterations to run
Return: A list containing mean and standard deviation for errors
'''
def run_bsgt (n, p, mode, maxIterations):
    k = len(p) # number of symbols
    error_list = [] # store error matrices across iterations
    error_matrix_sum = 0 # sum of error matrices across iterations
    
    for iteration in range(0, maxIterations):
        if debug and iteration: print("[DEBUG]")
        if debug: print("[DEBUG] Data set", iteration)
        
        error_matrix = []
        # Take 1000 subsets of samples for estimation
        # Only 1000 needed for plotting error versus sample size
        # Not optimized for n less than 1000
        for i in range(0, 1000):
            n_i = int(n / 1000 * (i + 1)) # current sample size
            if n_i == 0: continue
            symbol_cnt = sample_gen(n_i, p)
            q = estimate(n_i, p, symbol_cnt, mode)
            error = loss(p, q, ["L1", "KL_div"])
            error_matrix.append(error)
        
        # Calculate error mean over iterations
        error_matrix = np.asarray(error_matrix)
        error_list.append(error_matrix)
        error_matrix_sum += error_matrix
    error_matrix_mean = error_matrix_sum / maxIterations
    
    # Calculate error standard deviation over iterations
    error_matrix_sqsum = 0
    for iteration in range(0, len(error_list)):
        error_matrix_sqsum += np.square(error_list[iteration] - error_matrix_mean)
    error_matrix_std_dev = np.sqrt(error_matrix_sqsum / maxIterations)
    
    if debug: print ("[DEBUG] Error_matrix dimensions:", error_matrix_sum.shape)
    
    # returns a list consists of error matrix mean and its standard deviation
    # each row corresponds to a loss function, each col corresponds to a beta value
    return [error_matrix_mean.T, error_matrix_std_dev.T]

In [6]:
'''
Description: Plots the desicred graph with given data, selected plot mode and graph title
Param1: x is whatever the data on x axis should be (beta range for "Add-constant" mode,
        sample size for "Braess-Sauer" or "Good-Turing" mode.)
Param2: error_matrix_range is the data on y axis, which is always the error given by chosen error matrices.
Param3: mode indicates which estimator and what purpose the graph is for. Its value should be "Add-constant",
        "Braess-Sauer", or "Good-Turing".
Param4: title is the graph title
Return: None
'''
def plot(x, error_matrix_range, mode, title):
    # Graph trace definition
    trace1_upper = Scatter(
        name = "L1 Upper",
        showlegend = False,
        x = x,
        y = error_matrix_range[0][0] + error_matrix_range[1][0],
        mode = "lines",
        marker = dict(color = "444"),
        line = dict(width = 0),
        fillcolor = "rgba(231, 107, 243, 0.1)",
        fill = "tonexty")

    trace1 = Scatter(
        name = "L1",
        x = x,
        y = error_matrix_range[0][0],
        mode = "lines",
        line = dict(color = "rgb(231, 107, 243)"),
        fillcolor = "rgba(231, 107, 243, 0.1)",
        fill = "tonexty")

    trace1_lower = Scatter(
        name = "L1 Lower",
        showlegend = False,
        x = x,
        y = error_matrix_range[0][0] - error_matrix_range[1][0],
        marker = dict(color = "444"),
        line = dict(width = 0),
        mode = "lines")

    if mode == "Add-constant":
        trace2_upper = Scatter(
        name = "L2 Upper",
        showlegend = False,
        x = x,
        y = error_matrix_range[0][1] + error_matrix_range[1][1],
        mode = "lines",
        marker = dict(color = "444"),
        line = dict(width = 0),
        fillcolor = "rgba(0, 176, 246, 0.1)",
        fill = "tonexty")

    if mode == "Add-constant":
        trace2 = Scatter(
        name = "L2 squared",
        x = x,
        y = error_matrix_range[0][1],
        mode = "lines",
        line = dict(color = "rgb(0, 176, 246)"),
        fillcolor = "rgba(0, 176, 246, 0.1)",
        fill = "tonexty")

    if mode == "Add-constant":
        trace2_lower = Scatter(
        name = "L2 Lower",
        showlegend = False,
        x = x,
        y = error_matrix_range[0][1] - error_matrix_range[1][1],
        marker = dict(color = "444"),
        line = dict(width = 0),
        mode = "lines")
    
    if mode == "Add-constant":
        trace3_upper = Scatter(
        name = "chi Upper",
        showlegend = False,
        x = x,
        y = error_matrix_range[0][2] + error_matrix_range[1][2],
        mode = "lines",
        marker = dict(color = "444"),
        line = dict(width = 0),
        fillcolor = "rgba(0, 100, 80, 0.1)",
        fill = "tonexty")

    if mode == "Add-constant":
        trace3 = Scatter(
        name = "chi squared",
        x = x,
        y = error_matrix_range[0][2],
        mode = "lines",
        line = dict(color = "rgb(0, 100, 80)"),
        fillcolor = "rgba(0, 100, 80, 0.1)",
        fill = "tonexty")

    if mode == "Add-constant":
        trace3_lower = Scatter(
        name = "chi Lower",
        showlegend = False,
        x = x,
        y = error_matrix_range[0][2] - error_matrix_range[1][2],
        marker = dict(color = "444"),
        line = dict(width = 0),
        mode = "lines")
    
    if mode == "Add-constant":
        trace4_upper = Scatter(
        name = "KL Upper",
        showlegend = False,
        x = x,
        y = error_matrix_range[0][3] + error_matrix_range[1][3],
        mode = "lines",
        marker = dict(color = "444"),
        line = dict(width = 0),
        fillcolor = "rgba(117, 175, 150, 0.1)",
        fill = "tonexty")

    if mode == "Add-constant":
        trace4 = Scatter(
        name = "KL divergence",
        x = x,
        y = error_matrix_range[0][3],
        mode = "lines",
        line = dict(color = "rgb(117, 175, 150)"),
        fillcolor = "rgba(117, 175, 150, 0.1)",
        fill = "tonexty")

    if mode == "Add-constant":
        trace4_lower = Scatter(
        name = "KL Lower",
        showlegend = False,
        x = x,
        y = error_matrix_range[0][3] - error_matrix_range[1][3],
        marker = dict(color = "444"),
        line = dict(width = 0),
        mode = "lines")
        
    if mode == "Braess-Sauer" or mode == "Good-Turing":
        trace4_upper = Scatter(
        name = "KL Upper",
        showlegend = False,
        x = x,
        y = error_matrix_range[0][1] + error_matrix_range[1][1],
        mode = "lines",
        marker = dict(color = "444"),
        line = dict(width = 0),
        fillcolor = "rgba(117, 175, 150, 0.1)",
        fill = "tonexty")

    if mode == "Braess-Sauer" or mode == "Good-Turing":
        trace4 = Scatter(
        name = "KL divergence",
        x = x,
        y = error_matrix_range[0][1],
        mode = "lines",
        line = dict(color = "rgb(117, 175, 150)"),
        fillcolor = "rgba(117, 175, 150, 0.1)",
        fill = "tonexty")
    
    if mode == "Braess-Sauer" or mode == "Good-Turing":
        trace4_lower = Scatter(
        name = "KL Lower",
        showlegend = False,
        x = x,
        y = error_matrix_range[0][1] - error_matrix_range[1][1],
        marker = dict(color = "444"),
        line = dict(width = 0),
        mode = "lines")
    
    # Plotting configuration
    if mode == "Add-constant":
        title = title
        xaxis_title = "beta"
        yaxis_title = "Error"
        data = Data([trace1_lower, trace1, trace1_upper,
                     trace2_lower, trace2, trace2_upper,
                     trace3_lower, trace3, trace3_upper,
                     trace4_lower, trace4, trace4_upper
                    ])
    elif mode == "Braess-Sauer" or mode == "Good-Turing":
        title = title
        xaxis_title = "Sample size"
        yaxis_title = "Error"
        data = Data([trace1_lower, trace1, trace1_upper,
                     trace4_lower, trace4, trace4_upper
                    ])
    else:
        raise ValueError("Invalid mode")

    layout = Layout(
        xaxis = XAxis(
            title = xaxis_title,
            zeroline = False),
        yaxis = YAxis(
            title = yaxis_title,
            zeroline = False),
        title = title,
        height = 550
    )
    
    fig = Figure(data=data, layout=layout)
    plotly.offline.iplot(fig)

In [7]:
debug = 0
maxIterations = 100

# Add-constant estimator
# Given distribution and sample size, generate sample data,
# and plot error for estimation based on different beta
n = 10
p = [0.2, 0.1, 0.7]

beta_min = 0.1
beta_max = 10
beta_step = (beta_max - beta_min) / 1000
beta_arr = np.arange(beta_min, beta_max, beta_step)

error_matrix_range = run_add_constant (n, p, beta_arr, maxIterations)

# checkbox debug
# checkbox envelope
# sliders control beta range

lambd_widget = FloatSlider(min=0.00, max=1.00, step=0.01, value=0.50)
display(lambd_widget)
#display(widget_output)
#with widget_output:

# Rip off the last two columns for add-constant estimator
plot(beta_arr, np.array(error_matrix_range)[:,0:1000],"Add-constant",
    "Distribution over " + str(len(p)) + " elements: " + str(p))

#print("Distribution over", len(p),"elements:", p)
#print("n =", n, "with", maxIterations, "data set(s)")

In [8]:
debug = 0
maxIterations = 10

# Braess and Sauer estimator
# Given uniform and zipf distribution on 100 symbols,
# generate sample data and plot
mode = "Braess-Sauer"
n = 1000
sample_arr = np.arange(0, n, 1)

p = uniform(100)
error_matrix_range = run_bsgt (n, p, mode, maxIterations)
plot(sample_arr, np.array(error_matrix_range), mode,
     "BS estimator with uniform distribution on 100 symbols")

p = zipf(100, 1)
error_matrix_range = run_bsgt (n, p, mode, maxIterations)
plot(sample_arr, np.array(error_matrix_range), mode,
    "BS estimator with zipf1 distribution on 100 symbols")

p = zipf(100, 2)
error_matrix_range = run_bsgt (n, p, mode, maxIterations)
plot(sample_arr, np.array(error_matrix_range), mode,
    "BS estimator with zipf2 distribution on 100 symbols")

In [9]:
debug = 0
maxIterations = 10

# Good-Turing estimator
# Given uniform and zipf distribution on 100 symbols,
# generate sample data and plot
mode = "Good-Turing"
n = 1000
sample_arr = np.arange(0, n, 1)

p = uniform(100)
error_matrix_range = run_bsgt (n, p, mode, maxIterations)
plot(sample_arr, np.array(error_matrix_range), mode,
     "BS estimator with uniform distribution on 100 symbols")

p = zipf(100, 1)
error_matrix_range = run_bsgt (n, p, mode, maxIterations)
plot(sample_arr, np.array(error_matrix_range), mode,
    "BS estimator with zipf1 distribution on 100 symbols")

p = zipf(100, 2)
error_matrix_range = run_bsgt (n, p, mode, maxIterations)
plot(sample_arr, np.array(error_matrix_range), mode,
    "BS estimator with zipf2 distribution on 100 symbols")

Input:

Given a distribution

Given raw data

Given file (.csv and probably .mat)

Find max in L1 and L2, make the range twice of that

Do 100 or 1000 symbols distribution
uniform/zipf1/zipf2 distributions
100 iteration
Two plots: plot loss versus number of samples, with each estimator
6 plots in total: for 3 distributions, L1 & KL loss for each distribution